In [ ]:
!pip install -q pgmpy

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
from scipy.stats import ttest_1samp

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton 
from libpgm.discretebayesiannnetwork import DiscreteBayesianNetwork
from libpgm.pgmlearner import PGMLearner 

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir data
!cp '/content/drive/My Drive/pred-analit/GC-Predictive-Analytics-worksheet - Sheet1 - week5.csv' data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
df = pd.read_csv('data/GC-Predictive-Analytics-worksheet - Sheet1 - week5.csv')
#df.drop([9, 10])

In [ ]:
df

,cluster,Gender,Age,Ethnicity,Income,Work hours,Health condition,Education,Motivation,Attitude,Intention,Ownership,Unnamed: 12,1,ER,2,ER.1,3,ER.2,4,ER.3,5,ER.4,6,ER.5,7,ER.6,8,ER.7,9,ER.8,10,ER.9,11,ER.10,12,ER.11,13,ER.12,14,ER.13,15,ER.14,16 Recommendation,Actual ER,17 Recommendation,Actual ER.1,18 Recommendation,Estimated ER
0,0.0,Female,44.0,Caucasian,"150,000-199,999",50+ hrs/week,Good,4-year college degree,0.265623,0.526590,0.507169,0.559363,NaN,Phy,0.166667,diet,0.714286,diet,0.428571,diet,0.571429,diet,0.750000,diet,0.428571,diet,0.857143,diet,1.0,diet,1.0,diet,0.285714,diet,0.571429,diet,0.500000,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0,diet,0.0,NaN,NaN
1,1.0,Female,37.0,Asian,"100,000-149,999",16-35 hrs/week,Excellent,2-year college degree,0.386128,0.896973,0.736022,0.924759,NaN,diet,0.000000,diet,0.428571,diet,1.000000,diet,0.857143,diet,0.875000,diet,0.857143,diet,1.000000,diet,1.0,diet,1.0,diet,0.857143,diet,0.857143,diet,1.000000,diet,1.0,diet,0.857143,diet,0.428571,Phy,0,diet,0.0,NaN,NaN
2,2.0,Female,63.0,Caucasian,"0-24,999",50+ hrs/week,Excellent,2-year college degree,0.147511,0.593162,0.574159,0.928628,NaN,diet,0.000000,Phy,0.142857,Phy,0.000000,Phy,0.571429,Phy,0.000000,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.000000,diet,1.0,phy,1.000000,diet,0.333333,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Female,37.0,African American,"0-24,999",1-15 hrs/week,Fair,"Some college, but no degree",0.492041,0.586625,0.645084,0.799052,NaN,Phy,0.000000,diet,0.000000,phy,0.000000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,Female,62.0,African American,"50,000-99,999",1-15 hrs/week,Good,4-year college degree,0.594119,0.563496,0.659809,0.820454,NaN,diet,0.285714,diet,1.000000,Phy,0.470588,diet,0.900000,phy,0.916667,diet,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Male,59.0,Caucasian,"50,000-99,999",36-50 hrs/week,Fair,4-year college degree,0.760602,0.541951,0.750184,0.664756,NaN,phy,0.058824,diet,0.500000,diet,0.500000,phy,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,Female,28.0,Asian,"50,000-99,999",50+ hrs/week,Good,4-year college degree,0.653543,0.731372,0.751441,0.976597,NaN,phy,0.250000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Male,60.0,Asian,"150,000-199,999",50+ hrs/week,Good,Some graduate work,0.736708,0.675436,0.760342,0.974514,NaN,phy,0.166667,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,Male,57.0,African American,"25,000-49,000",50+ hrs/week,Excellent,Some graduate work,0.550593,0.765072,0.800639,0.804968,NaN,diet,0.833333,Phy,0.857143,Phy,0.857143,Phy,1.000000,Phy,1.000000,Phy,0.714286,Phy,1.000000,Phy,1.0,Phy,1.0,Phy,1.000000,Phy,1.000000,Phy,0.833333,Phy,1.0,Phy,1.000000,Phy,1.000000,NaN,DROPPED,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.drop([9, 10])

,cluster,Gender,Age,Ethnicity,Income,Work hours,Health condition,Education,Motivation,Attitude,Intention,Ownership,Unnamed: 12,1,ER,2,ER.1,3,ER.2,4,ER.3,5,ER.4,6,ER.5,7,ER.6,8,ER.7,9,ER.8,10,ER.9,11,ER.10,12,ER.11,13,ER.12,14,ER.13,15,ER.14,16 Recommendation,Actual ER,17 Recommendation,Actual ER.1,18 Recommendation,Estimated ER
0,0.0,Female,44.0,Caucasian,"150,000-199,999",50+ hrs/week,Good,4-year college degree,0.265623,0.526590,0.507169,0.559363,NaN,Phy,0.166667,diet,0.714286,diet,0.428571,diet,0.571429,diet,0.750000,diet,0.428571,diet,0.857143,diet,1.0,diet,1.0,diet,0.285714,diet,0.571429,diet,0.500000,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0,diet,0.0,NaN,NaN
1,1.0,Female,37.0,Asian,"100,000-149,999",16-35 hrs/week,Excellent,2-year college degree,0.386128,0.896973,0.736022,0.924759,NaN,diet,0.000000,diet,0.428571,diet,1.000000,diet,0.857143,diet,0.875000,diet,0.857143,diet,1.000000,diet,1.0,diet,1.0,diet,0.857143,diet,0.857143,diet,1.000000,diet,1.0,diet,0.857143,diet,0.428571,Phy,0,diet,0.0,NaN,NaN
2,2.0,Female,63.0,Caucasian,"0-24,999",50+ hrs/week,Excellent,2-year college degree,0.147511,0.593162,0.574159,0.928628,NaN,diet,0.000000,Phy,0.142857,Phy,0.000000,Phy,0.571429,Phy,0.000000,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.000000,diet,1.0,phy,1.000000,diet,0.333333,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Female,37.0,African American,"0-24,999",1-15 hrs/week,Fair,"Some college, but no degree",0.492041,0.586625,0.645084,0.799052,NaN,Phy,0.000000,diet,0.000000,phy,0.000000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,Female,62.0,African American,"50,000-99,999",1-15 hrs/week,Good,4-year college degree,0.594119,0.563496,0.659809,0.820454,NaN,diet,0.285714,diet,1.000000,Phy,0.470588,diet,0.900000,phy,0.916667,diet,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Male,59.0,Caucasian,"50,000-99,999",36-50 hrs/week,Fair,4-year college degree,0.760602,0.541951,0.750184,0.664756,NaN,phy,0.058824,diet,0.500000,diet,0.500000,phy,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,Female,28.0,Asian,"50,000-99,999",50+ hrs/week,Good,4-year college degree,0.653543,0.731372,0.751441,0.976597,NaN,phy,0.250000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Male,60.0,Asian,"150,000-199,999",50+ hrs/week,Good,Some graduate work,0.736708,0.675436,0.760342,0.974514,NaN,phy,0.166667,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,Male,57.0,African American,"25,000-49,000",50+ hrs/week,Excellent,Some graduate work,0.550593,0.765072,0.800639,0.804968,NaN,diet,0.833333,Phy,0.857143,Phy,0.857143,Phy,1.000000,Phy,1.000000,Phy,0.714286,Phy,1.000000,Phy,1.0,Phy,1.0,Phy,1.000000,Phy,1.000000,Phy,0.833333,Phy,1.0,Phy,1.000000,Phy,1.000000,NaN,DROPPED,NaN,NaN,NaN,NaN
11,1.0,Male,40.0,HIspanic,"0-24,999",50+ hrs/week,Fair,4-year college degree,0.689156,0.647716,0.738081,0.975890,NaN,diet,0.000000,diet,0.000000,diet,0.285714,,0.000000,diet,0.125000,diet,0.428571,diet,0.000000,diet,0.0,diet,0.0,diet,0.000000,diet,0.000000,diet,0.000000,diet,0.0,diet,0.000000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rec_idxs = [str(i) for i in range(1, 15)]
er_idxs = ['ER']
er_idxs.extend(['ER.{}'.format(i) for i in range(1, 14)])
rec_idxs.append('16 Recommendation ')
rec_idxs.append('17 Recommendation')
er_idxs.append('Actual ER')
er_idxs.append('Actual ER.1')

In [ ]:
df.columns

Index(['cluster', 'Gender', 'Age', 'Ethnicity', 'Income', 'Work hours',
       'Health condition', 'Education', 'Motivation', 'Attitude', 'Intention',
       'Ownership', 'Unnamed: 12', '1', 'ER', '2', 'ER.1', '3', 'ER.2', '4',
       'ER.3', '5', 'ER.4', '6', 'ER.5', '7', 'ER.6', '8', 'ER.7', '9', 'ER.8',
       '10', 'ER.9', '11', 'ER.10', '12', 'ER.11', '13', 'ER.12', '14',
       'ER.13', '15', 'ER.14', '16 Recommendation ', 'Actual ER',
       '17 Recommendation', 'Actual ER.1', '18 Recommendation',
       'Estimated ER'],
      dtype='object')

In [ ]:
rrecs = []
eers = []
iids = []
for i in range(len(df)):
  recs = []
  ers = []
  for rec, er in zip(rec_idxs, er_idxs):
      recomendation = df[rec][i]
      try:
        engagement = float(df[er][i])
      except ValueError:
        engagement = 0.0
      if pd.isna(recomendation):
        continue
      elif recomendation == 'Phy':
        recomendation = 0
      else:
        recomendation = 1
      recs.append(recomendation)
      ers.append(engagement)
  rrecs.append(recs)
  eers.append(ers)
  gender = df.Gender[i]
  hours = df['Work hours'][i]
  hmap = {'1-15 hrs/week': 0.2, '16-35 hrs/week': 0.4, '36-50 hrs/week': 0.6, '50+ hrs/week': 1.0}
  if hours in hmap:
    hours = hmap[hours]
  else:
    hours = 0.0
  if gender == 'Female':
    gender = 0
  else:
    gender = 1
  motivation = df.Motivation[i]	
  if pd.isna(motivation):
    motivation = 0.0
  attitude = df.Attitude[i]
  if pd.isna(attitude):
    attitude = 0
  intention = df.Intention[i]
  if pd.isna(intention):
    intention = 0
  ownership = df.Ownership[i]
  if pd.isna(ownership):
    ownership = 0
  iids.append(dict(age = df.Age[i] / 82,
                   gender = gender,  
                   motivation = motivation,	
                   attitude = attitude, 
                   intention = intention,
                   ownership = ownership,
                   hours = hours))

In [ ]:
X = []
y = []
for i in range(len(rrecs)):
  for j in range(len(rrecs[i]) - 2):
    r1 = rrecs[i][j]
    r2 = rrecs[i][j+1]
    # X.append([eers[i][j], r2 - r1, r1, r2, iids[i]['age']])
    X.append([eers[i][j], r1, r2, r2-r1, iids[i]['age'], iids[i]['motivation']]) #iids[i]['attitude']])
    y.append(eers[i][j+1])

In [ ]:
np.unique(y)

array([0.        , 0.125     , 0.14285714, 0.28571429, 0.42857143,
       0.47058824, 0.5       , 0.57142857, 0.71428571, 0.75      ,
       0.83333333, 0.85714286, 0.875     , 0.9       , 0.91666667,
       1.        ])

# Method evaluation

### regular features

In [ ]:
models = []
models.append(('Linear', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('RegTree', RandomForestRegressor()))

# evaluate each model in turn
for name, model in models:
    cv_results = cross_val_score(model, X, y, cv=KFold(n_splits=5, shuffle=True), scoring='r2')
    print("{}: {:.3f} ({})".format(name, cv_results.mean(), cv_results))

Linear: 0.679 ([0.87126999 0.73799811 0.6576607  0.77929864 0.34661542])
Ridge: 0.669 ([0.70525571 0.55811253 0.89359894 0.52351708 0.66642926])
Lasso: -0.232 ([-0.86262201 -0.06655991 -0.00880634 -0.12744547 -0.09288968])
Elastic: -0.120 ([-0.1807298  -0.26404856 -0.01094246 -0.00418345 -0.14112854])
RegTree: 0.656 ([0.52926253 0.57763073 0.79758297 0.69788859 0.67757576])


### polynomial features

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)

In [ ]:
models = []
models.append(('Linear', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('RegTree', RandomForestRegressor()))

# evaluate each model in turn
for name, model in models:
    cv_results = cross_val_score(model, X_, y, cv=KFold(n_splits=5, shuffle=True), scoring='r2')
    print("{}: {:.3f} ({})".format(name, cv_results.mean(), cv_results))

Linear: 0.651 ([0.54432772 0.85915356 0.61787163 0.69151615 0.5440076 ])
Ridge: 0.690 ([0.55658275 0.73341766 0.61994351 0.73298133 0.80628135])
Lasso: -0.051 ([-0.0932321  -0.00274372 -0.04892001 -0.03978454 -0.06979086])
Elastic: -0.083 ([-0.00627273 -0.00796537 -0.09481302 -0.01682365 -0.28843341])
RegTree: 0.592 ([0.54969254 0.6647057  0.25892652 0.72583341 0.76290271])


# Weekly prediction

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)
model = RandomForestRegressor()
model.fit(X_, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
for i in [0, 1, 2, 3, 4, 5, 6, 7, 8]:
    r1 = rrecs[i][-1]
    diet_er = model.predict(poly.transform(np.array([eers[i][-1], r1, 1 - r1, 1, iids[i]['age'], iids[i]['motivation']]).reshape(1, -1)))
    phy_er = model.predict(poly.transform(np.array([eers[i][-1], r1, 0 - r1, 0, iids[i]['age'], iids[i]['motivation']]).reshape(1, -1)))
    if diet_er > phy_er:
      print('id: {} recomendation: {} er: {}'.format(i, 'diet', diet_er[0]))
    else:
      print('id: {} recomendation: {} er: {}'.format(i, 'Phy', phy_er[0]))

id: 0 recomendation: diet er: 0.2415992063581334
id: 1 recomendation: diet er: 0.2368571428688001
id: 2 recomendation: Phy er: 0.515927093058139
id: 3 recomendation: diet er: 0.08571428571799998
id: 4 recomendation: Phy er: 0.7874066831811419
id: 5 recomendation: Phy er: 0.6820642923774894
id: 6 recomendation: diet er: 0.027142857143
id: 7 recomendation: diet er: 0.4504840303556355
id: 8 recomendation: Phy er: 0.9342276949206366


In [ ]:
sum_diet = 0
sum_phy = 0
diet_n = 0
phy_n = 0
for ers_, rec_ in zip(eers, rrecs):
  for i in range(len(ers_)):
    if rec_[i] == 1:
      sum_diet += ers_[i]
      diet_n += 1
    else:
      sum_phy += ers_[i]
      phy_n += 1

diet = sum_diet / diet_n
phy = sum_phy / phy_n

In [ ]:
diet

0.32818004979566656

In [ ]:
phy

0.2400511928913793